In [2]:
import n2n4m.preprocessing as preprocessing
import pandas as pd
import numpy as np

In [3]:
# Set random seed
np.random.seed(42)

In [4]:
# Load data
MINERAL_DATASET_PATH = "/home/rp1818/N2N4M/N2N4M/data/full_pixel_dataset.json"
BLAND_DATASET_PATH = "/home/rp1818/N2N4M/N2N4M/data/full_bland_pixel_dataset.json"
mineral_dataset = preprocessing.load_dataset(MINERAL_DATASET_PATH)
bland_dataset = preprocessing.load_dataset(BLAND_DATASET_PATH)

In [5]:
# Sample the bland pixels, as don't use all of them
num_bland_pixels = 150_000
num_bland_images = bland_dataset["Image_Name"].nunique()
samples_per_image = num_bland_pixels // num_bland_images
bland_dataset_sample = bland_dataset.groupby("Image_Name").apply(lambda x: x.sample(min(len(x), samples_per_image), random_state=42)).reset_index(drop=True)

/tmp/ipykernel_328934/2633476336.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  bland_dataset_sample = bland_dataset.groupby("Image_Name").apply(lambda x: x.sample(min(len(x), samples_per_image), random_state=42)).reset_index(drop=True)


In [6]:
# Apply preprocessing
dataset = pd.concat([mineral_dataset, bland_dataset_sample], ignore_index=True).reset_index(drop=True)
dataset = preprocessing.expand_dataset(dataset)
dataset = preprocessing.drop_bad_bands(dataset, preprocessing.PLEBANI_WAVELENGTHS)
dataset = preprocessing.impute_bad_values(dataset)
dataset = preprocessing.impute_atmospheric_artefacts(dataset)

/raid/home/rp1818/N2N4M/N2N4M/n2n4m/preprocessing.py:200: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.update(class_subset)
/raid/home/rp1818/N2N4M/N2N4M/n2n4m/preprocessing.py:200: FutureWarning: Downcasting behavior in Series and DataFrame methods 'where', 'mask', and 'clip' is deprecated. In a future version this will not infer object dtypes or cast all-round floats to integers. Instead call result.infer_objects(copy=False) for object inference, or cast round floats explicitly. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset.update(class_subset)
/raid/home/rp1818/N2N4M/N2N4M/

In [9]:
# Generate training samples and split into train and testing data
noise_dataset = preprocessing.generate_noisy_pixels(dataset.iloc[:,3:], random_seed=42)
input_target_dataset = pd.concat([dataset, noise_dataset], axis=1)
train_set, test_set = preprocessing.train_test_split(input_target_dataset, bland_pixels=True)
train_set, validation_set = preprocessing.train_validation_split(train_set, bland_pixels=True)

In [10]:
# Split into features, targets and ancillary data
X_train, y_train, ancillary_train = preprocessing.split_features_targets_anciliary(train_set)
X_test, y_test, ancillary_test = preprocessing.split_features_targets_anciliary(test_set)
X_validation, y_validation, ancillary_validation = preprocessing.split_features_targets_anciliary(validation_set)


In [11]:
# Standardise the features
X_train, feature_scaler = preprocessing.standardise(X_train, method="RobustScaler")
X_test, _ = preprocessing.standardise(X_test, method="RobustScaler", scaler=feature_scaler)
X_validation, _ = preprocessing.standardise(X_validation, method="RobustScaler", scaler=feature_scaler)


In [15]:
from joblib import dump
dump(feature_scaler, "n2n4m_feature_scaler.joblib")

['n2n4m_feature_scaler.joblib']